In [1]:
from os.path import exists
import requests
from typing import *

import numpy as np
import pandas as pd
from tqdm.notebook import trange, tqdm

In [2]:
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as stats

# Notebook 3: Optimization and Regression

## Problem 1: A Fully Connected and Single-Layer Neural Network

Suppose we are fitting a regression model to a dataset $(x_i, y_i)_{1 \leq i \leq N}$
\begin{align*}
p(y^i|x^i; A, \mu) & = \mathcal{N}(f(Ax^i + \mu), 1) \\
p(y |x; A, \mu) & = \prod_{i=1}^N p(y^i|x^i; A, \mu)
\end{align*}
where
1. the inputs $x^i \in \mathbb{R}^d$ are d-D vectors
2. $A$ is a $1 \times d$ matrix of weights
$$
A = \begin{pmatrix}
a_{11} & \dots & a_{1d} 
\end{pmatrix}
$$
3. $f: \mathbb{R} \rightarrow \mathbb{R}$ is some function
4. $x$ is a $Nxd$ matrix where row $i$ contains $x^i$
$$
x = \begin{pmatrix}
- & x^1 & -\\
\vdots & \vdots & \vdots\\
- & x^N & -
\end{pmatrix}
$$
5. $y$ is a vector of values to regress against
$$
y = \begin{pmatrix}
y^1 \\
\vdots \\
y^N
\end{pmatrix}
$$
6. $\mu \in \mathbb{R}$ is an offset

### Problem 1a

Implement the conditional density:
$$
p(y |x; A, \mu) = \prod_{i=1}^N p(y^i|x^i; A, \mu)
$$

In [3]:
# scout resources to verify correctness of the implementation.

x = np.array([[1,2,3], [4,5,6], [7,8,9], [0,0,0], [1,1,1], [2,2,2]])

A = np.array([0.25,0.5,0.75]) #True value
Mu = np.array([5]) #True value

A_rand = np.random.rand(3)
Mu_rand = np.random.rand(1)

def f(x): return 2*x
def grad_f(x): return np.array([2])

y = f(A @ x.T + Mu)

y

array([17., 26., 35., 10., 13., 16.])

In [4]:
def model1_density(f: Callable[[float], float],
                   A: np.ndarray,
                   mu: float) -> Callable[[np.ndarray, np.ndarray], float]:
    # Inputs:
    #     f is some function
    #     A is a 1xd matrix
    #     mu is a float indicating the constant offset
    #
    # Outputs:
    #     Return the density as a function of the dataset x and y
    
    def logdensity(x: np.ndarray, y: np.ndarray) -> float:
        # Inputs:
        #     x is a Nxd matrix where each row is a datapoint
        #
        # Outputs:
        #     y is a length n vector to evaluate the density on
        
        #return np.sum(np.log((density_f(x, y))))
        
        normals = [ stats.norm(loc = f(A @ x[i].T + mu), scale = 1) for i in range(x.shape[0]) ]
        density_f = [ normals[i].pdf(y[i]) for i in range(len(y)) ]
        
        return np.prod(density_f)

    return lambda x, y: logdensity(x, y)

In [5]:
model1_density(f, A, Mu)(x, y)

0.004031441804149938

### Problem 1b

Derive and implement
$$
\frac{\partial}{\partial A} p(y |x; A, \mu)
$$
and
$$
\frac{\partial}{\partial \mu} p(y |x; A, \mu) \,.
$$

In [6]:
import code

def grad_A(f: Callable[[float], float],
           grad_f: Callable[[float], float],
           A: np.ndarray,
           mu: float,
           x: np.ndarray,
           y: np.ndarray) -> np.ndarray:
    
       # Inputs:
       #     f is some function
       #     grad_f is the gradient of f
       #     A is a 1xd matrix
       #     mu is a float indicating the constant offset
       #     x is a nxd matrix of inputs
       #     y is a length n vector of regression outputs
       #
       # Outputs:
       #     Return the gradient of the conditional density w.r.t. the matrix A, which should be a 1xd matrix.

       #delA = lambda x, y: ( x * np.e**(0.5 * (y - f(A @ x + mu))**2) * (f(A @ x + mu) - y) * grad_f(A @ x + mu) ) / np.sqrt(2 * np.pi)
       #grad = [ delA(x[i], y[i]) for i in range(x.shape[0]) ]

       log_delA_xi = lambda xi, yi: -1 * xi * grad_f(A @ xi + mu) * (yi - f(A @ xi + mu))
       log_delA = [ log_delA_xi(x[i], y[i]) for i in range(x.shape[0]) ]

       #log_delA_xi = lambda xi, yi: [ xi.T @ ((yi - (f(A * xi[d] + mu))) * (grad_f(A * xi[d] + mu))) for d in range(xi.shape[0]) ]
       #log_gradA = [ np.sum(log_delA_xi(x[i], y[i])) for i in range(x.shape[0]) ]

       log_gradA = np.sum(log_delA, axis=0)

       return log_gradA

       pass

In [7]:
def grad_mu(f: Callable[[float], float],
            grad_f: Callable[[float], float],
            A: np.ndarray,
            mu: float,
            x: np.ndarray,
            y: np.ndarray) -> float:
        # Inputs:
        #     f is some function
        #     grad_f is the gradient of f
        #     A is a 1xd matrix
        #     mu is a float indicating the constant offset
        #     x is a nxd matrix of inputs
        #     y is a length n vector of regression outputs
        #
        # Outputs:
        #     Return the gradient of the conditional density w.r.t. the offset mu, which should be a float.

        #delmu = lambda x, y: (np.e**(0.5 * (y - f(A @ x + mu))**2) * (f(A @ x + mu) - y) * grad_f(A @ x + mu)) / np.sqrt(2 * np.pi) #is it y or x?
        #grad = [ delmu(x[i], y[i]) for i in range(x.shape[0]) ]

        #log_delmu_xi = lambda x, y: (y - (f(A @ x + mu)) * (1 - grad_f(A @ x + mu))) #is it y or x?
        #log_gradmu = [ log_delmu_xi(x[i], y[i]) for i in range(x.shape[0]) ]

        log_delmu_xi = lambda xi, yi: -1 * grad_f(A @ xi + mu) * (yi - f(A @ xi + mu))
        log_delmu = [ log_delmu_xi(x[i], y[i]) for i in range(x.shape[0]) ]

        log_gradmu = np.sum(log_delmu, axis=0)

        return log_gradmu

In [8]:
grad_A(f, grad_f, A_rand, Mu_rand, x, y).round(2), grad_mu(f, grad_f, A_rand, Mu_rand, x, y).round(2)

(array([-370.33, -446.39, -522.44]), array([-127.74]))

### Problem 1c

Write a function that solves for the weights by finding the approximate minimum of the conditional density, i.e., solve
$$
\operatorname{argmin}_{A, \mu} -p(y | x; A, \mu)
$$
with **stochastic gradient descent**. Hint: you may want to use the negative log-likelihood trick.

In [9]:
def solve_for_weights1(f: Callable[[float], float],
                      grad_f: Callable[[float], float],
                      x: np.ndarray, 
                      y: np.ndarray,
                      A: np.ndarray,
                      mu: float,
                      initial_A: np.ndarray,
                      initial_mu: np.ndarray,
                      step_size: float,
                      batch_size: int) -> Tuple[np.ndarray, float]:
    # Inputs:
    #     f is some function
    #     grad_f is the gradient of f
    #     A is a 1xd matrix
    #     mu is a float indicating the constant offset
    #     initial_A is a 1xd matrix containing the initial guess of A for stochastic gradient descent
    #     initial_mu is a float containing the initial guess of mu for stochastic gradient descent
    #     step_size is a the step size of stochastic gradient descent
    #     batch_size is the batch size of stochastic gradient descent    
    # 
    # Outputs:
    #     Return the density as a function of the dataset x and y
    #     
    #     !!do argmin of the negative log likelihood of p(y|x, A, mu)
    
    best_A = initial_A
    best_mu = initial_mu
    
    #best_A = np.zeros((x.shape[1]))
    #best_mu = 0

    for i in range(100):
        perm = np.random.permutation(len(y))
        shuffled_x = x[perm]
        shuffled_y = y[perm]
    
    for j in range(0, x.shape[0], batch_size):
        
        batch_x = shuffled_x[j*batch_size:(j+1)*batch_size,:]
        batch_y = shuffled_y[j*batch_size:(j+1)*batch_size]
        
        step_A = step_size * grad_A(f, grad_f, best_A, best_mu, batch_x, batch_y)
        step_mu = step_size * grad_mu(f, grad_f, best_A, best_mu, batch_x, batch_y)

        new_density = model1_density(f, best_A - step_A, best_mu - step_mu)(batch_x, batch_y)
        old_density = model1_density(f, best_A, best_mu)(batch_x, batch_y)

        if new_density > old_density:
            best_A = best_A - step_A
            best_mu = best_mu - step_mu

    return best_A, best_mu

In [10]:
#initial_A = np.random.rand(x.shape[1])
#initial_mu = np.random.rand(1)

#initial_A = np.zeros((x.shape[1]))
#initial_mu = 0

initial_A = np.array([0.5, 0.75, 1])
initial_mu = 7

step_size = 2e-3
batch_size = 1

gd_A, gd_Mu = solve_for_weights1(f, grad_f, x, y, A, Mu, initial_A, initial_mu, step_size, batch_size)

print("A from SGD:" , gd_A.round(2))
print("Mu from SGD:" , gd_Mu.round(2))

print("\nProbability Density after SGD:" , model1_density(f, gd_A, gd_Mu)(x, y))

A from SGD: [0.09 0.29 0.49]
Mu from SGD: [6.91]

Probability Density after SGD: 3.1600389862386185e-19


## Problem 2: A Fully Connected and Two-Layer Neural Network

Suppose we are fitting a regression model to a dataset $(x_i, y_i)_{1 \leq i \leq N}$
\begin{align*}
p(y^i|x^i; A_1, \mu_1, A_2, \mu_2) & = \mathcal{N}(f(A_2 f(A_1x^i + \mu_1) + \mu_2), 1) \\
p(y |x; A_1, \mu_1, A_2, \mu_2) & = \prod_{i=1}^N p(y^i|x^i; A_1, \mu_1, A_2, \mu_2)
\end{align*}
where
1. the inputs $x^i \in \mathbb{R}^d$ are d-D vectors
2. $A_1$ is a $m \times d$ matrix of weights and $A_2$ is a $1 \times m$ matrix of weights.
$$
A = \begin{pmatrix}
a_{11} & \dots & a_{1d} 
\end{pmatrix}
$$
3. $\mu_1 \in \mathbb{R}^m$ is a vector of weights and $\mu_2 \in \mathbb{R}$ is a weight.
4. $f: \mathbb{R} \rightarrow \mathbb{R}$ is some function
5. $x$ is a $Nxd$ matrix where row $i$ contains $x^i$
$$
x = \begin{pmatrix}
- & x^1 & -\\
\vdots & \vdots & \vdots\\
- & x^N & -
\end{pmatrix}
$$
6. $y$ is a vector of values to regress against
$$
y = \begin{pmatrix}
y^1 \\
\vdots \\
y^N
\end{pmatrix}
$$

### Problem 2a

Implement the conditional density:
$$
p(y |x; A_1, \mu_1, A_2, \mu_2) = \prod_{i=1}^N p(y^i|x^i; A_1, \mu_1, A_2, \mu_2)
$$

In [11]:
# scout resources to verify correctness of the implementation.

x = np.array([[1,2,3], [4,5,6], [7,8,9], [0,0,0], [1,1,1], [2,2,2]])

A1 = np.array([[0.25,0.5,0.75], [0.1, 0.2, 0.3], [0.2, 0.3, 0.4]]) #True value, m = 3
A2 = np.array([0.5,0.75,1]) #True value
A1_rand = np.random.rand(3,3)
A2_rand = np.random.rand(3)

Mu1 = np.array([0.5,0.6,0.7]) #True value
Mu2 = np.array([0.9]) #True value
Mu1_rand = np.random.rand(3)
Mu2_rand = np.random.rand(1)

def f(x): return 2*x
def grad_f(x): return np.array([2])

y = np.asarray([ f(A2 @ f(A1 @ x[i] + Mu1) + Mu2) for i in range(x.shape[0]) ])

y

array([[26.6],
       [51.8],
       [77. ],
       [ 7.4],
       [15.8],
       [24.2]])

In [12]:
def model2_density(f: Callable[[float], float],
                   A1: np.ndarray,
                   mu1: float,
                   A2: np.ndarray,
                   mu2: float) -> Callable[[np.ndarray, np.ndarray], float]:
    # Inputs:
    #     f is some function
    #     A1 is a mxd matrix
    #     mu1 is a length m vector
    #     A2 is a 1xm matrix
    #     mu2 is a float indicating the constant offset
    #
    # Outputs:
    #     Return the density as a function of the dataset x and y
    
    def density(x: np.ndarray, y: np.ndarray) -> float:
        # Inputs:
        #     x is a Nxd matrix where each row is a datapoint
        #
        # Outputs:
        #     y is a length n vector to evaluate the density on\

        normals = [ stats.norm(loc=f(A2 @ f(A1 @ x[i] + mu1) + mu2), scale = 1) for i in range(x.shape[0]) ]
        density_f = [ normals[i].pdf(y[i]) for i in range(len(y)) ]
        
        return np.prod(density_f)

    return lambda x,y: density(x,y)

In [13]:
model2_density(f, A1, Mu1, A2, Mu2)(x, y)

0.004031441804149938

### Problem 2b

Derive and implement
$$
\frac{\partial}{\partial A_1} p(y |x; A_1, \mu_1, A_2, \mu_2)
$$
and
$$
\frac{\partial}{\partial \mu_1} p(y |x; A_1, \mu_1, A_2, \mu_2) \,.
$$

In [14]:
def grad_A1(f: Callable[[float], float],
            grad_f: Callable[[float], float],
            A1: np.ndarray,
            mu1: float,
            A2: np.ndarray,
            mu2: float,
            x: np.ndarray,
            y: np.ndarray) -> np.ndarray:
        # Inputs:
        #     f is some function
        #     grad_f is the gradient of f
        #     A1 is a mxd matrix of weights
        #     mu1 is a vector of constant offsets
        #     A2 is a 1xm matrix of weights
        #     mu2 is a float that is a constant offset
        #     x is a nxd matrix of inputs
        #     y is a length n vector of regression outputs
        #
        # Outputs:
        #     Return the gradient of the conditional density w.r.t. the matrix A, which should be a 1xd matrix.

        log_delA1_xi = lambda xi, yi: -1 * xi * (A2 * grad_f(A1 @ xi + Mu1) * grad_f(A2 @ f(A1 @ xi + Mu1) + Mu2) * (yi - f(A2 @ f(A1 @ xi + Mu1) + Mu2)))
        log_delA1 = [ log_delA1_xi(x[i], y[i]) for i in range(x.shape[0]) ]
        
        log_gradA1 = np.sum(log_delA1, axis = 0)

        return log_gradA1

In [15]:
def grad_mu1(f: Callable[[float], float],
            grad_f: Callable[[float], float],
            A1: np.ndarray,
            mu1: float,
            A2: np.ndarray,
            mu2: float,
            x: np.ndarray,
            y: np.ndarray) -> np.ndarray:
        # Inputs:
        #     f is some function
        #     grad_f is the gradient of f
        #     A1 is a mxd matrix of weights
        #     mu1 is a vector of constant offsets
        #     A2 is a 1xm matrix of weights
        #     mu2 is a float that is a constant offset
        #     x is a nxd matrix of inputs
        #     y is a length n vector of regression outputs
        #
        # Outputs:
        #     Return the gradient of the conditional density w.r.t. the matrix A, which should be a 1xd matrix.

        log_delmu1_xi = lambda xi, yi: -1 * A2 * grad_f(A1 @ xi + Mu1) * grad_f(A2 @ f(A1 @ xi + Mu1) + Mu2) * (yi - f(A2 @ f(A1 @ xi + Mu1) + Mu2))
        log_delmu1 = [ log_delmu1_xi(x[i], y[i]) for i in range(x.shape[0]) ]
        
        log_gradmu1 = np.sum(log_delmu1, axis = 0)

        return log_gradmu1

In [16]:
grad_A1(f, grad_f, A1_rand, Mu1_rand, A2_rand, Mu2_rand, x, y).round(2), grad_mu1(f, grad_f, A1_rand, Mu1_rand, A2_rand, Mu2_rand, x, y).round(2)

(array([528.19, 718.31, 653.99]), array([104.15, 121.32,  96.59]))

### Problem 2c

Derive and implement
$$
\frac{\partial}{\partial A_2} p(y |x; A_1, \mu_1, A_2, \mu_2)
$$
and
$$
\frac{\partial}{\partial \mu_2} p(y |x; A_1, \mu_1, A_2, \mu_2) \,.
$$

In [17]:
def grad_A2(f: Callable[[float], float],
            grad_f: Callable[[float], float],
            A1: np.ndarray,
            mu1: float,
            A2: np.ndarray,
            mu2: float,
            x: np.ndarray,
            y: np.ndarray) -> np.ndarray:
        # Inputs:
        #     f is some function
        #     grad_f is the gradient of f
        #     A1 is a mxd matrix of weights
        #     mu1 is a vector of constant offsets
        #     A2 is a 1xm matrix of weights
        #     mu2 is a float that is a constant offset
        #     x is a nxd matrix of inputs
        #     y is a length n vector of regression outputs
        #
        # Outputs:
        #     Return the gradient of the conditional density w.r.t. the matrix A, which should be a 1xd matrix.

        log_delA2_xi = lambda xi, yi: -1 * f(A1 @ xi + Mu1 * grad_f(A2 @ f(A1 @ xi + Mu1) + Mu2) * (yi - f(A2 @ f(A1 @ xi + Mu1) + Mu2)))
        log_delA2 = [ log_delA2_xi(x[i], y[i]) for i in range(x.shape[0]) ]
        
        log_gradA2 = np.sum(log_delA2, axis = 0)

        return log_gradA2

In [18]:
def grad_mu2(f: Callable[[float], float],
             grad_f: Callable[[float], float],
             A1: np.ndarray,
             mu1: float,
             A2: np.ndarray,
             mu2: float,
             x: np.ndarray,
             y: np.ndarray) -> float:
    # Inputs:
    #     f is some function
    #     grad_f is the gradient of f
    #     A1 is a mxd matrix of weights
    #     mu1 is a vector of constant offsets
    #     A2 is a 1xm matrix of weights
    #     mu2 is a float that is a constant offset
    #     x is a nxd matrix of inputs
    #     y is a length n vector of regression outputs
    #
    # Outputs:
    #     Return the gradient of the conditional density w.r.t. the matrix A, which should be a 1xd matrix.

    log_delmu2_xi = lambda xi, yi: -1 * grad_f(A2 @ f(A1 @ xi + Mu1) + Mu2) * (yi - f(A2 @ f(A1 @ xi + Mu1) + Mu2))
    log_delmu2 = [ log_delmu2_xi(x[i], y[i]) for i in range(x.shape[0]) ]
    
    log_gradmu2 = np.sum(log_delmu2, axis = 0)

    return log_gradmu2

In [19]:
grad_A2(f, grad_f, A1_rand, Mu1_rand, A2_rand, Mu2_rand, x, y).round(2), grad_mu2(f, grad_f, A1_rand, Mu1_rand, A2_rand, Mu2_rand, x, y).round(2)

(array([13.62, 69.68, 55.05]), array([85.84]))

### Problem 2d

Write a function that solves for the weights by finding the approximate minimum of the conditional density, i.e., solve
$$
\operatorname{argmin}_{A_1, \mu_1, A_2, \mu_2} -p(y | x; A_1, \mu_1, A_2, \mu_2)
$$
with **stochastic gradient descent**. Hint: you may want to use the negative log-likelihood trick.

In [20]:
def solve_for_weights2(f: Callable[[float], float],
                      grad_f: Callable[[float], float],
                      x: np.ndarray,
                      y: np.ndarray,
                      A1: np.ndarray,
                      mu1: np.ndarray,
                      A2: np.ndarray,
                      mu2: float,
                      initial_A1: np.ndarray,
                      initial_mu1: np.ndarray,
                      initial_A2: np.ndarray,
                      initial_mu2: float,
                      step_size: float,
                      batch_size: int) -> Tuple[np.ndarray, float]:
    # Inputs:
    #     f is some function
    #     grad_f is the gradient of f
    #     A1 is a mxd matrix of weights
    #     mu1 is a vector of constant offsets
    #     A2 is a 1xm matrix of weights
    #     mu2 is a float that is a constant offset
    #     initial_A1 is a mxd matrix containing the initial guess of A1 for stochastic gradient descent
    #     initial_mu1 is a length d vector containing the initial guess of mu1 for stochastic gradient descent
    #     initial_A2 is a 1xm matrix containing the initial guess of A2 for stochastic gradient descent
    #     initial_mu2 is a float containing the initial guess of mu2 for stochastic gradient descent
    #     step_size is a the step size of stochastic gradient descent
    #     batch_size is the batch size of stochastic gradient descent    
    #
    # Outputs:
    #     Return the density as a function of the dataset x and y

    best_A1 = initial_A1
    best_mu1 = initial_mu1
    best_A2 = initial_A2
    best_mu2 = initial_mu2

    for i in range(100):
    # 1. Shuffle the dataset randomly
        perm2 = np.random.permutation(len(y))
        shuffled_x = x[perm2]
        shuffled_y = y[perm2]
    
    # 2. Break the dataset into batches of size batch_size
    for j in range(0, x.shape[0], batch_size):
        
        # 3. Create a batch
        batch_x = shuffled_x[j*batch_size:(j+1)*batch_size,:]
        batch_y = shuffled_y[j*batch_size:(j+1)*batch_size]
        
        # 4. Perform gradient descent using a batch
        step_A1 = step_size * grad_A1(f, grad_f, best_A1, best_mu1, best_A2, best_mu2, batch_x, batch_y)
        step_mu1 = step_size * grad_mu1(f, grad_f, best_A1, best_mu1, best_A2, best_mu2, batch_x, batch_y)
        step_A2 = step_size * grad_A2(f, grad_f, best_A1, best_mu1, best_A2, best_mu2, batch_x, batch_y)
        step_mu2 = step_size * grad_mu2(f, grad_f, best_A1, best_mu1, best_A2, best_mu2, batch_x, batch_y)

        new_density = model2_density(f, best_A1 - step_A1, best_mu1 - step_mu1, best_A2 - step_A2, best_mu2 - step_mu2)(x, y)
        current_density = model2_density(f, best_A1, best_mu1, best_A2, best_mu2)(x, y)

        if new_density > current_density:
            best_A1 = best_A1 - step_A1
            best_mu1 = best_mu1 - step_mu1
            best_A2 = best_A2 - step_A2
            best_mu2 = best_mu2 - step_mu2
    
    return best_A1, best_mu1, best_A2, best_mu2

In [21]:
step_size = 1e-3
batch_size = 1

initial_A1 = np.array([[0.5,0.6,0.7], [0.25, 0.3, 0.5], [0.6, 0.7, 0.8]]) #True value, m = 3
initial_A2 = np.array([0.1,0.2,0.3]) #True value
initial_Mu1 = np.array([0.8,0.9,1]) #True value
initial_Mu2 = np.array([1]) #True value

gdA1, gdMu1, gdA2, gdMu2 = solve_for_weights2(f, grad_f, x, y, A1, Mu1, A2, Mu2, initial_A1, initial_Mu1, initial_A2, initial_Mu2, step_size, batch_size)

print("\nA1 from SGD:" , gdA1.round(2))
print("Mu1 from SGD:" , gdMu1.round(2))
print("A2 from SGD:" , gdA2.round(2))
print("Mu2 from SGD:" , gdMu2.round(2))

print("\nProbability Density after SGD:" , model2_density(f, gdA1, gdMu1, gdA2, gdMu2)(x, y))


A1 from SGD: [[0.56 0.74 0.95]
 [0.31 0.44 0.75]
 [0.66 0.84 1.05]]
Mu1 from SGD: [0.82 0.94 1.06]
A2 from SGD: [0.22 0.33 0.46]
Mu2 from SGD: [1.09]

Probability Density after SGD: 0.00010606986048196353


### Problem 3: Multi-output Regression

Suppose we are fitting a regression model to a dataset $(x_i, y_i)_{1 \leq i \leq N}$
\begin{align*}
p(y^i|x^i; A, \mu, \Sigma) & = \mathcal{N}(f(Ax^i + \mu), \Sigma) \\
p(y |x; A, \mu, \Sigma) & = \prod_{i=1}^N p(y^i|x^i; A, \mu, \Sigma)
\end{align*}
where
1. the inputs $x^i \in \mathbb{R}^d$ are d-D vectors
2. $A$ is a $2 \times d$ matrix of weights
$$
A = \begin{pmatrix}
a_{11} & \dots & a_{1d} \\
a_{21} & \dots & a_{2d}
\end{pmatrix}
$$
3. $f: \mathbb{R} \rightarrow \mathbb{R}$ is some function
4. $x$ is a $Nxd$ matrix where row $i$ contains $x^i$
$$
x = \begin{pmatrix}
- & x^1 & -\\
\vdots & \vdots & \vdots\\
- & x^N & -
\end{pmatrix}
$$
5. $y$ is a $Nx2$ matrix of values to regress against
$$
y = \begin{pmatrix}
y^1_1 & y^1_2\\
\vdots \\
y^N_1 & y^N_2\\
\end{pmatrix}
$$
6. $\Sigma$ is a covariance matrix
$$
\begin{pmatrix}
\sigma_{11} & \sigma_{12} \\
\sigma_{21} & \sigma_{22}
\end{pmatrix}
$$


### Problem 3a

Suppose we are solving for
$$
\operatorname{argmin}_{A, \mu} -p(y | x; A, \mu, \Sigma) \,.
$$

Derive the associated loss function.

In [48]:
# product of all the normal densities

def model3_density(f: Callable[[float], float],
                   A: np.ndarray,
                   sigma: np.ndarray,
                   mu: np.ndarray) -> Callable[[np.ndarray, np.ndarray], float]:
    
    def density(x: np.ndarray, y: np.ndarray) -> float:

        mvnorms = [ stats.multivariate_normal(mean=f(A @ x[i].T + mu), cov=sigma) for i in range(len(y)) ]
        density_f = [ mvnorms[i].pdf(y[i]) for i in range(y.shape[0]) ]
        
        return np.prod(density_f, axis=0)

    return lambda x,y: density(x,y)

### Problem 3b

Compare and contrast the situation when $\Sigma$ is an identity matrix and when it is not.

In [52]:
mu = np.array([0.,0]) #True value
identity = np.eye(2)

L = np.array([[sp.stats.norm().rvs(), 0.0], [sp.stats.norm().rvs(), sp.stats.norm().rvs()]])
sigma = L @ L.transpose()

x = np.array([[1,3], [3,5], [5,7], [7,9], [9,11]])
#y = np.array([2,4,6,8,10])

A = np.array([[0.1,0.2], [0.2, 0.3]])

def f(x): return 2*x

sigma

array([[0.20600476, 0.29424385],
       [0.29424385, 1.2242376 ]])

In [53]:
id_distribution = [ stats.multivariate_normal(mean= f(A @ x[i].T), cov=identity) for i in range(x.shape[0]) ]
ys1 = np.array([ id_distribution[i].rvs(size=1) for i in range(x.shape[0]) ])

nonid_distribution = [ stats.multivariate_normal(mean= f(A @ x[i].T), cov=sigma) for i in range(x.shape[0]) ]
ys2 = np.array([ nonid_distribution[i].rvs(size=1) for i in range(x.shape[0]) ])

ys1, ys2

(array([[-0.06744104,  3.26690482],
        [ 2.99861857,  4.93275506],
        [ 2.32434514,  6.13683571],
        [ 5.63531998,  8.12180746],
        [ 4.45093891, 10.04339148]]),
 array([[1.70943973, 1.96319819],
        [2.42805384, 4.26902711],
        [4.16174029, 5.56301653],
        [4.36007953, 7.79341432],
        [6.33974817, 9.83586476]]))

In [54]:
model3_density(f, A, identity, mu)(x, ys1), model3_density(f, A, sigma, mu)(x, ys2)

(8.145337377632025e-07, 0.0003741124346843708)

An identity covariance would have a symettric distribution over all the dimension w.r.t. the mean vector.

The non-identity covariance matrix leads to a skewed distribution. This leads to a more/less dense distribution around the mean.

A non-identity covariance matrix is a matrix where the variances are not all the same. This means that the variables are not all independent of each other. While an identity covariance means all the variables are idependent.